
# End-to-End ML (Kaggle Dataset)

This notebook shows how to go from **business question → production approach** using a Kaggle dataset.
We will predict claim outcomes and/or flag high-cost claims, following a practical ML workflow:

1. Frame the business & ML problem  
2. Load the data  
3. Explore & visualize  
4. Prepare & engineer features  
5. Train baseline models (regression & classification)  
6. Fine-tune with cross-validation  
7. Present results (metrics, feature importances, ROC)  
8. Discuss deployment: launch, monitor, maintain

> You can upload the **full Kaggle CSV**. For quick testing, a small sample CSV is included.



## 1) Look at the Big Picture — Frame the Problem

**Business questions (examples):**
- *Regression:* What will the **ultimate incurred claim cost** be?  
- *Classification:* Will the claim be **high-cost** (e.g., top 10%)?

**Why it matters:**
- Better reserving and budgeting.
- Early triage and investigation of risky claims.
- Faster processing of low-risk claims.

**ML setup:**
- Supervised learning on historical claims.
- Target: `UltimateIncurredClaimCost` (regression) or `HighCost` (classification).



## 2) Get the Data — Load the CSV

- **Option A:** Upload your Kaggle CSV in the left Files pane, then set `csv_path` below.
- **Option B:** Use the included mini sample to verify the pipeline works end-to-end.


In [ ]:
import pandas as pd

# Set to your uploaded Kaggle CSV file name, e.g.:
# csv_path = "/content/<your_kaggle_file>.csv"
# For demo, we'll start with the small sample:
csv_path = "datasets/easy_peasy.csv"

df = pd.read_csv(csv_path)
df.head()


## 3) Explore & Visualize
We'll parse dates, engineer simple time features, and look at distributions and relationships.


In [ ]:
import numpy as np

# Parse ISO timestamps
for col in ["DateTimeOfAccident", "DateReported"]:
    df[col] = pd.to_datetime(df[col], errors="coerce")

# Feature: Days between accident and report
df["DaysToReport"] = (df["DateReported"] - df["DateTimeOfAccident"]).dt.days

# Time features
df["AccidentYear"] = df["DateTimeOfAccident"].dt.year
df["AccidentMonth"] = df["DateTimeOfAccident"].dt.month

df[["DateTimeOfAccident","DateReported","DaysToReport","AccidentYear","AccidentMonth"]].head()

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt

# Target distribution (if available)
if "UltimateIncurredClaimCost" in df.columns:
    plt.figure()
    df["UltimateIncurredClaimCost"].hist(bins=40)
    plt.xlabel("UltimateIncurredClaimCost")
    plt.ylabel("Count")
    plt.title("Target Distribution (Raw)")
    plt.show()

    plt.figure()
    import numpy as np
    np.log1p(df["UltimateIncurredClaimCost"]).hist(bins=40)
    plt.xlabel("log1p(UltimateIncurredClaimCost)")
    plt.ylabel("Count")
    plt.title("Target Distribution (Log)")
    plt.show()

# Relationship: WeeklyWages vs Ultimate Cost
if {"WeeklyWages","UltimateIncurredClaimCost"} <= set(df.columns):
    plt.figure()
    plt.scatter(df["WeeklyWages"], df["UltimateIncurredClaimCost"])
    plt.xlabel("WeeklyWages")
    plt.ylabel("UltimateIncurredClaimCost")
    plt.title("WeeklyWages vs Ultimate Cost")
    plt.show()

In [ ]:
# Correlation (numerical columns only)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if "UltimateIncurredClaimCost" in df.columns:
    corr = df[num_cols].corr(numeric_only=True)
    corr_top = corr["UltimateIncurredClaimCost"].sort_values(ascending=False).head(10)
    display(corr_top.to_frame("corr_with_target"))
else:
    print("UltimateIncurredClaimCost not found; skipping correlation view.")


## 4) Prepare the Data
- Select features and target.
- Handle categorical variables via one-hot encoding.
- Scale numerical features for linear models.
- Split into train/test.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Classification target (high cost = top 10%)
if "UltimateIncurredClaimCost" in df.columns:
    threshold = df["UltimateIncurredClaimCost"].quantile(0.9)
    df["HighCost"] = (df["UltimateIncurredClaimCost"] >= threshold).astype(int)

drop_cols = ["ClaimNumber","DateTimeOfAccident","DateReported"]
target_reg = "UltimateIncurredClaimCost"
target_clf = "HighCost" if "HighCost" in df.columns else None

feature_df = df.drop(columns=[c for c in drop_cols if c in df.columns])
X = feature_df.drop(columns=[t for t in [target_reg, target_clf] if t in feature_df.columns])
y_reg = df[target_reg] if target_reg in df.columns else None
y_clf = df[target_clf] if target_clf else None

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
num_cols = X.select_dtypes(exclude=["object"]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

X_train, X_test, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)
if y_clf is not None:
    _, _, y_train_clf, y_test_clf = train_test_split(X, y_clf, test_size=0.2, random_state=42)

len(X_train), len(X_test)



## 5) Select & Train Baseline Models
- **Regression:** Linear Regression and Random Forest Regressor
- **Classification:** Logistic Regression and Random Forest Classifier


In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import numpy as np

# Regression baselines
reg_lin = Pipeline([("prep", preprocessor), ("model", LinearRegression())])
reg_lin.fit(X_train, y_train_reg)
pred_lin = reg_lin.predict(X_test)
print("Regression — Linear: RMSE={:.2f}, MAE={:.2f}, R2={:.3f}".format(
    np.sqrt(mean_squared_error(y_test_reg, pred_lin)),
    mean_absolute_error(y_test_reg, pred_lin),
    r2_score(y_test_reg, pred_lin)
))

reg_rf = Pipeline([("prep", preprocessor), ("model", RandomForestRegressor(random_state=42))])
reg_rf.fit(X_train, y_train_reg)
pred_rf = reg_rf.predict(X_test)
print("Regression — RF:     RMSE={:.2f}, MAE={:.2f}, R2={:.3f}".format(
    np.sqrt(mean_squared_error(y_test_reg, pred_rf)),
    mean_absolute_error(y_test_reg, pred_rf),
    r2_score(y_test_reg, pred_rf)
))

In [ ]:
# Classification baselines
if "HighCost" in df.columns and df["HighCost"].nunique() == 2:
    clf_log = Pipeline([("prep", preprocessor), ("model", LogisticRegression(max_iter=200))])
    clf_log.fit(X_train, y_train_clf)
    proba_log = clf_log.predict_proba(X_test)[:,1]
    pred_log = (proba_log >= 0.5).astype(int)
    print("Classification — Logistic AUC: {:.3f}".format(roc_auc_score(y_test_clf, proba_log)))
    print(classification_report(y_test_clf, pred_log))

    clf_rf = Pipeline([("prep", preprocessor), ("model", RandomForestClassifier(random_state=42))])
    clf_rf.fit(X_train, y_train_clf)
    proba_rf = clf_rf.predict_proba(X_test)[:,1]
    pred_rf = (proba_rf >= 0.5).astype(int)
    print("Classification — RF AUC: {:.3f}".format(roc_auc_score(y_test_clf, proba_rf)))
    print(classification_report(y_test_clf, pred_rf))

    # ROC curves
    plt.figure()
    fpr_log, tpr_log, _ = roc_curve(y_test_clf, proba_log)
    fpr_rf, tpr_rf, _ = roc_curve(y_test_clf, proba_rf)
    plt.plot(fpr_log, tpr_log, label="Logistic")
    plt.plot(fpr_rf, tpr_rf, label="RandomForest")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curves")
    plt.legend()
    plt.show()
else:
    print("Classification target not available; skipping classification section.")


## 6) Fine-Tune Your Model (Randomized Search)
Small search for Random Forest — both regression and classification.


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rf_reg = Pipeline([("prep", preprocessor), ("model", RandomForestRegressor(random_state=42))])
param_dist_reg = {
    "model__n_estimators": [100, 200, 400],
    "model__max_depth": [None, 10, 20, 40],
    "model__min_samples_split": [2, 5, 10]
}
# Use neg_mean_squared_error for broad version compatibility; we'll report RMSE on the test set below.
search_reg = RandomizedSearchCV(
    rf_reg,
    param_distributions=param_dist_reg,
    n_iter=6,
    scoring="neg_mean_squared_error",
    cv=3,
    random_state=42,
    n_jobs=-1
)
search_reg.fit(X_train, y_train_reg)
best_reg = search_reg.best_estimator_
print("Best reg params:", search_reg.best_params_)

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

pred_best = best_reg.predict(X_test)
print("Best RF Test RMSE:", np.sqrt(mean_squared_error(y_test_reg, pred_best)))

In [ ]:
if "HighCost" in df.columns and df["HighCost"].nunique() == 2:
    rf_clf = Pipeline([("prep", preprocessor), ("model", RandomForestClassifier(random_state=42))])
    param_dist_clf = {
        "model__n_estimators": [100, 200, 400],
        "model__max_depth": [None, 10, 20, 40],
        "model__min_samples_split": [2, 5, 10]
    }
    search_clf = RandomizedSearchCV(rf_clf, param_distributions=param_dist_clf, n_iter=6,
                                    scoring="roc_auc", cv=3, random_state=42, n_jobs=-1)
    search_clf.fit(X_train, y_train_clf)
    best_clf = search_clf.best_estimator_
    print("Best clf params:", search_clf.best_params_)

    from sklearn.metrics import roc_auc_score
    proba_best = best_clf.predict_proba(X_test)[:,1]
    print("Best RF Test AUC:", roc_auc_score(y_test_clf, proba_best))


## 7) Present Your Solution — Drivers & Insights
We show feature importances (top 10) for tuned models.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_feature_names(prep, num_cols, cat_cols):
    num_features = list(num_cols)
    ohe = None
    try:
        ohe = prep.named_transformers_["cat"]
    except Exception:
        for name, trans, cols in prep.transformers_:
            if name == "cat":
                ohe = trans
                break
    if hasattr(ohe, "get_feature_names_out"):
        cat_features = list(ohe.get_feature_names_out(cat_cols))
    else:
        cat_features = []
    return num_features + cat_features

num_cols = X.select_dtypes(exclude=["object"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

# Regression importances
feature_names = get_feature_names(best_reg.named_steps["prep"], num_cols, cat_cols)
importances = best_reg.named_steps["model"].feature_importances_
order = np.argsort(importances)[::-1][:10]

plt.figure()
plt.barh([feature_names[i] for i in order][::-1], importances[order][::-1])
plt.xlabel("Importance")
plt.title("Top 10 Features (Regression RF)")
plt.show()

In [ ]:
# Classification importances
if "HighCost" in df.columns and df["HighCost"].nunique() == 2:
    feature_names = get_feature_names(best_clf.named_steps["prep"], num_cols, cat_cols)
    importances = best_clf.named_steps["model"].feature_importances_
    order = np.argsort(importances)[::-1][:10]

    plt.figure()
    plt.barh([feature_names[i] for i in order][::-1], importances[order][::-1])
    plt.xlabel("Importance")
    plt.title("Top 10 Features (Classification RF)")
    plt.show()


## 8) Launch, Monitor, and Maintain — Production Approach

**How to ship this at WCF:**
1. Save the tuned pipeline using `joblib` (it includes preprocessing + model).
2. Expose a prediction endpoint or batch job to score new claims.
3. Store predictions with timestamps; route high-risk claims for human review.
4. Monitor model drift and performance; retrain monthly/quarterly.
5. Add governance: access controls, audit logs, bias monitoring by demographic segments.



## **Extra Analysis Ideas**

In this section, we go beyond the basic regression/classification models and explore additional analytical techniques:

1. **Clustering:** Segment claims based on injury description keyword embeddings to find patterns in claim types and costs.
2. **Time-Series Trend Analysis:** Explore how claim costs and frequency evolve over time.
3. **Scenario Simulation:** Estimate potential savings if the reporting delay is reduced.


In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

# Select the claim description keyword columns
keyword_cols = [col for col in df.columns if col.startswith("ClaimDescriptionKeyword_")]
X_keywords = df[keyword_cols]

# Run KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42)
df['ClaimCluster'] = kmeans.fit_predict(X_keywords)

# Cluster analysis: average cost per cluster
cluster_costs = df.groupby('ClaimCluster')['UltimateIncurredClaimCost'].mean()
print("Average cost per cluster:\n", cluster_costs)

# Visualize clusters using first two PCA components
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
components = pca.fit_transform(X_keywords)

plt.scatter(components[:,0], components[:,1], c=df['ClaimCluster'], cmap='viridis', alpha=0.6)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('Claim Clusters (based on keywords)')
plt.show()

In [ ]:
# Time-series: average cost per accident year
df['AccidentYear'] = df['DateTimeOfAccident'].dt.year
yearly_costs = df.groupby('AccidentYear')['UltimateIncurredClaimCost'].mean()

yearly_costs.plot(marker='o')
plt.ylabel('Average Claim Cost')
plt.title('Average Claim Cost by Accident Year')
plt.grid(True)
plt.show()

# Claim frequency per year
yearly_counts = df.groupby('AccidentYear').size()
yearly_counts.plot(marker='s', color='orange')
plt.ylabel('Number of Claims')
plt.title('Number of Claims by Accident Year')
plt.grid(True)
plt.show()

In [ ]:
# Scenario: Reduce DaysToReport to <= 7 days for all claims
df['DaysToReport'] = (df['DateReported'] - df['DateTimeOfAccident']).dt.days
baseline_avg_cost = df['UltimateIncurredClaimCost'].mean()

# Assume that for late-reported claims (>7 days), cost could be reduced by 5%
df['SimulatedCost'] = df.apply(lambda row: row['UltimateIncurredClaimCost'] * 0.95 if row['DaysToReport'] > 7 else row['UltimateIncurredClaimCost'], axis=1)

simulated_avg_cost = df['SimulatedCost'].mean()
savings = baseline_avg_cost - simulated_avg_cost

print(f"Baseline avg cost: {baseline_avg_cost:,.2f}")
print(f"Simulated avg cost (report <= 7 days): {simulated_avg_cost:,.2f}")
print(f"Estimated savings per claim: {savings:,.2f}")
print(f"Estimated annual savings (assuming {len(df)} claims/year): {savings * len(df):,.2f}")